# Analysis of Under Flow Assays

   Copyright 2015-2025, University of Bern,
    Data Science Lab and Theodor Kocher Institute,
    M. Vladymyrov

   Licensed under the Apache License, Version 2.0 (the "License");
   you may not use this file except in compliance with the License.
   You may obtain a copy of the License at

       http://www.apache.org/licenses/LICENSE-2.0

   Unless required by applicable law or agreed to in writing, software
   distributed under the License is distributed on an "AS IS" BASIS,
   WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
   See the License for the specific language governing permissions and
   limitations under the License.


In [1]:
import sys
lib_root = r'g:\IVFCA\UFMTrack'
sys.path.append(lib_root) # see the GitHub referenced in the paper

from ana_gui import configure_analysis_gui, study_ui

# Configure and synchronize segmented datasets

In [ ]:
configure_analysis_gui()

# Perform a study

In [2]:
study_ui()